In [1]:
"""Uses Stable-Baselines3 to train agents in the Knights-Archers-Zombies environment using SuperSuit vector envs.

This environment requires using SuperSuit's Black Death wrapper, to handle agent death.

For more information, see https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html

Author: Elliot (https://github.com/elliottower)
"""
from __future__ import annotations

import glob
import os
import time
import numpy as np
import time

import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy, MlpPolicy

from pettingzoo.mpe import simple_tag_v3
# from pettingzoo.butterfly import knights_archers_zombies_v10


def train(env_fn, steps: int = 100, seed: int | None = 0, **env_kwargs):
    # Train a single model to play as each agent in an AEC environment
    env = env_fn.parallel_env(**env_kwargs)

    # Add black death wrapper so the number of agents stays constant
    # MarkovVectorEnv does not support environments with varying numbers of active agents unless black_death is set to True
    env = ss.black_death_v3(env)

    # Pre-process using SuperSuit
    visual_observation = False #not env.unwrapped.continuous_actions not env.unwrapped.vector_state
    if visual_observation:
        # If the observation space is visual, reduce the color channels, resize from 512px to 84px, and apply frame stacking
        env = ss.color_reduction_v0(env, mode="B")
        env = ss.resize_v1(env, x_size=84, y_size=84)
        env = ss.frame_stack_v1(env, 3)

    env.reset(seed=seed)

    print(f"Starting training on {str(env.metadata['name'])}.")
    env = ss.multiagent_wrappers.pad_observations_v0(env)
    env = ss.pettingzoo_env_to_vec_env_v1(env)
    env = ss.concat_vec_envs_v1(env, 8, num_cpus=1, base_class="stable_baselines3")

    # Use a CNN policy if the observation space is visual
    model = PPO(
        CnnPolicy if visual_observation else MlpPolicy,
        env,
        verbose=3,
        batch_size=256,
    )

    model.learn(total_timesteps=steps)

    model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    print("Model has been saved.")

    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")

    env.close()

In [2]:
# Set vector_state to false in order to use visual observations (significantly longer training time)
env_kwargs = dict(num_good=1, num_adversaries=3, num_obstacles=2, max_cycles=25, continuous_actions=False )
# max_cycles=100, max_zombies=4, vector_state=True
# Train a model (takes ~5 minutes on a laptop CPU)
env_fn = simple_tag_v3
train(env_fn, steps=10000, seed=0, **env_kwargs)

Starting training on simple_tag_v3.
Using cuda device


C:\Users\aron_\.conda\envs\deeplearning\lib\site-packages\pettingzoo\utils\conversions.py:252: UserWarning: The base environment `simple_tag_v3` does not have a `render_mode` defined.
  warnings.warn(
C:\Users\aron_\.conda\envs\deeplearning\lib\site-packages\pettingzoo\utils\conversions.py:132: UserWarning: The base environment `aec_observation_lambda<simple_tag_v3>` does not have a `render_mode` defined.
  warnings.warn(


------------------------------
| time/              |       |
|    fps             | 4477  |
|    iterations      | 1     |
|    time_elapsed    | 14    |
|    total_timesteps | 65536 |
------------------------------
Model has been saved.
Finished training on simple_tag_v3.


In [3]:

def eval(env_fn, num_games: int = 10000, render_mode: str | None = None, **env_kwargs):
    # Evaluate a trained agent vs a random agent
    env = env_fn.env(render_mode=render_mode, **env_kwargs)

    # Pre-process using SuperSuit
    # if visual_observation = False #not env.unwrapped.continuous_actions
    if visual_observation:
        # If the observation space is visual, reduce the color channels, resize from 512px to 84px, and apply frame stacking
        env = ss.color_reduction_v0(env, mode="B")
        env = ss.resize_v1(env, x_size=84, y_size=84)
        env = ss.frame_stack_v1(env, 3)

    print(
        f"\nStarting evaluation on {str(env.metadata['name'])} (num_games={num_games}, render_mode={render_mode})"
    )

    try:
        latest_policy = max(
            glob.glob(f"{env.metadata['name']}*.zip"), key=os.path.getctime
        )
    except ValueError:
        print("Policy not found.")
        exit(0)

    model = PPO.load(latest_policy)

    rewards = {agent: 0 for agent in env.possible_agents}

    # Note: we evaluate here using an AEC environments, to allow for easy A/B testing against random policies
    # For example, we can see here that using a random agent for archer_0 results in less points than the trained agent
    for i in range(num_games):
        env.reset(seed=i)
        env.action_space(env.possible_agents[0]).seed(i)
        
        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()
            if agent == 'agent_0':
                obs=np.append(obs, [0,0])
            #print(obs)
            if render_mode== 'human':
                time.sleep(0.01)
            for agent in env.agents:
                rewards[agent] += env.rewards[agent]

            if termination or truncation:
                break
            else:
                if agent == env.possible_agents[0]:
                    act = env.action_space(agent).sample()
                else:
                    act = model.predict(obs, deterministic=True)[0]
            env.step(act)
    env.close()

    avg_reward = sum(rewards.values()) / len(rewards.values())
    avg_reward_per_agent = {
        agent: rewards[agent] / num_games for agent in env.possible_agents
    }
    print(f"Avg reward: {avg_reward}")
    print("Avg reward per agent, per game: ", avg_reward_per_agent)
    print("Full rewards: ", rewards)
    return avg_reward


In [ ]:
# Set vector_state to false in order to use visual observations (significantly longer training time)
env_kwargs = dict(num_good=1, num_adversaries=3, num_obstacles=2, max_cycles=25, continuous_actions=False)
eval(env_fn, num_games=10, render_mode=None, **env_kwargs)

In [ ]:
from __future__ import annotations

import glob
import os
import time
import numpy as np
import time

import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy, MlpPolicy

from pettingzoo.mpe import simple_tag_v3
env_fn = simple_tag_v3
print(env_fn)

In [ ]:
# Set vector_state to false in order to use visual observations (significantly longer training time)
env_kwargs = dict(num_good=1, num_adversaries=3, num_obstacles=2, max_cycles=200, continuous_actions=False )
eval(env_fn, 100, 'human', **env_kwargs)